# Goto https://globus-sdk-python.readthedocs.io/en/stable/tutorial.html#first-steps

# Finish Step 1 and 2

# Save CLIENT_ID

In [ ]:
import sys
!{sys.executable} -m pip install globus-sdk

In [ ]:
import globus_sdk

CLIENT_ID = "b6b3ffb1-bf21-4a3e-ad91-a91a8f4e71ab"

client = globus_sdk.NativeAppAuthClient(CLIENT_ID)
client.oauth2_start_flow(refresh_tokens=True)

authorize_url = client.oauth2_get_authorize_url()
print("Please go to this URL and login: {0}".format(authorize_url))

auth_code = input("Please enter the code you get after login here: ").strip()
token_response = client.oauth2_exchange_code_for_tokens(auth_code)

globus_auth_data = token_response.by_resource_server["auth.globus.org"]
globus_transfer_data = token_response.by_resource_server["transfer.api.globus.org"]

# This is the long term transfer token (sensitive info)

In [ ]:
long_term_transfer_token = globus_transfer_data["refresh_token"]
long_term_transfer_token

# Test data transfer from Expanse/Comet to CJW NFS (optional)

In [ ]:
# Test long term transfer token
## see: https://globus-sdk-python.readthedocs.io/en/stable/examples/advanced_transfer.html

import logging
from globus_sdk import GlobusAPIError, NetworkError

# putting logger objects named by the module name into the module-level
# scope is a common best practice -- for more details, you should look
# into the python logging documentation
logger = logging.getLogger(__name__)


def retry_globus_function(func, retries=5, func_name="<func>"):
    """
    Define what it means to retry a "Globus Function", some function or
    method which produces Globus SDK errors on failure.
    """

    def actually_retry():
        """
        Helper: run the next retry
        """
        return retry_globus_function(func, retries=(retries - 1), func_name=func_name)

    def check_for_reraise():
        """
        Helper: check if we should reraise an error
                logs an error message on reraise
                must be run inside an exception handler
        """
        if retries < 1:
            logger.error("Retried {} too many times.".format(func_name))
            raise

    try:
        return func()
    except NetworkError:
        # log with exc_info=True to capture a full stacktrace as a
        # debug-level log
        logger.debug(
            ("Globus func {} experienced a network error".format(func_name)),
            exc_info=True,
        )
        check_for_reraise()
    except GlobusAPIError:
        # again, log with exc_info=True to capture a full stacktrace
        logger.warn(
            ("Globus func {} experienced a network error".format(func_name)),
            exc_info=True,
        )
        check_for_reraise()

    # if we reach this point without returning or erroring, retry
    return actually_retry()

def submit_transfer_with_retries(transfer_client, transfer_data):
    # create a function with no arguments, for our retry handler
    def locally_bound_func():
        return transfer_client.submit_transfer(transfer_data)

    return retry_globus_function(locally_bound_func, func_name="submit_transfer")

In [ ]:
client = globus_sdk.NativeAppAuthClient(CLIENT_ID)
authorizer = globus_sdk.RefreshTokenAuthorizer(
    long_term_transfer_token, client,
)
tc = globus_sdk.TransferClient(authorizer=authorizer)
# Expanse/Comet
source_endpoint_id = "b256c034-1578-11eb-893e-0a5521ff3f4b"
# CJW-NFS-On-Jetstream
destination_endpoint_id = "4a874b58-ebde-11eb-ab62-d195c983855c"

tdata = globus_sdk.TransferData(tc, source_endpoint_id,
                                 destination_endpoint_id,
                                 label="Test Transfer",
                                 sync_level="checksum")
tdata.add_item("/~/comet_folder", "/~/vol_b/notebook_home_data/drew/",
                recursive=True)
transfer_result = submit_transfer_with_retries(tc, tdata)
task_id = transfer_result["task_id"]
print("task_id =", transfer_result["task_id"])

In [ ]:
# see: https://github.com/globus/globus-jupyter-notebooks
response = tc.get_task(task_id)
response.data["status"] 

# Go to https://app.globus.org/ to check job activity

# Put token on CyberGIS-Compute Server

# Clean up this notebook to remove all sensitive info